In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Load the data

In [2]:
df = pd.read_excel('../data/Online_Retail.xlsx')


## Show Top 5 Records

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Shape of the Dataset

In [4]:
df.shape

(541909, 8)

## Dataset Information
* InvoiceNo --> Categorical
* StockCode --> Categorical
* Description --> Categorical
* Quantity --> Integer
* Invoice Date --> Date
* UnitPrice --> Continuous
* CustomerID --> Categorical
* Country --> Categorical


## Preprocess the data

In [5]:
# Convert 'InvoiceDate' to datetime format for time-related features

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
# Calculate the total bill for each transaction

df['Total_Bill'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_Bill
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [7]:
# Aggregate data at Customer Level

customer_df = df.groupby('CustomerID').agg(
    Total_Bill_Size=('Total_Bill', 'sum'),
    First_Purchase=('InvoiceDate', 'min'),
    Last_Purchase=('InvoiceDate', 'max'),
    Most_Common_Location=('Country', lambda x: x.mode()[0]),
    Top_Item=('StockCode', lambda x: x.value_counts().idxmax())
)
customer_df.head()

,Total_Bill_Size,First_Purchase,Last_Purchase,Most_Common_Location,Top_Item
CustomerID,,,,,
12346.0,0.00,2011-01-18 10:01:00,2011-01-18 10:17:00,United Kingdom,23166
12347.0,4310.00,2010-12-07 14:57:00,2011-12-07 15:52:00,Iceland,22375
12348.0,1797.24,2010-12-16 19:09:00,2011-09-25 13:13:00,Finland,POST
12349.0,1757.55,2011-11-21 09:51:00,2011-11-21 09:51:00,Italy,23112
12350.0,334.40,2011-02-02 16:01:00,2011-02-02 16:01:00,Norway,21908


In [8]:
# Calculate Price Interval in days

customer_df['Purchase_Interval_Days'] = (customer_df['Last_Purchase'] - customer_df['First_Purchase']).dt.days
customer_df.head()

,Total_Bill_Size,First_Purchase,Last_Purchase,Most_Common_Location,Top_Item,Purchase_Interval_Days
CustomerID,,,,,,
12346.0,0.00,2011-01-18 10:01:00,2011-01-18 10:17:00,United Kingdom,23166,0
12347.0,4310.00,2010-12-07 14:57:00,2011-12-07 15:52:00,Iceland,22375,365
12348.0,1797.24,2010-12-16 19:09:00,2011-09-25 13:13:00,Finland,POST,282
12349.0,1757.55,2011-11-21 09:51:00,2011-11-21 09:51:00,Italy,23112,0
12350.0,334.40,2011-02-02 16:01:00,2011-02-02 16:01:00,Norway,21908,0


In [9]:
#  Reset index to have 'CustomerID' as a column

customer_df.reset_index(inplace=True)
customer_df.head()

,CustomerID,Total_Bill_Size,First_Purchase,Last_Purchase,Most_Common_Location,Top_Item,Purchase_Interval_Days
0,12346.0,0.00,2011-01-18 10:01:00,2011-01-18 10:17:00,United Kingdom,23166,0
1,12347.0,4310.00,2010-12-07 14:57:00,2011-12-07 15:52:00,Iceland,22375,365
2,12348.0,1797.24,2010-12-16 19:09:00,2011-09-25 13:13:00,Finland,POST,282
3,12349.0,1757.55,2011-11-21 09:51:00,2011-11-21 09:51:00,Italy,23112,0
4,12350.0,334.40,2011-02-02 16:01:00,2011-02-02 16:01:00,Norway,21908,0


## Normalize the features

In [10]:
# Standardize the 'Total_Bill_Size' feature

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
customer_df['Total_Bill_Size_Scaled'] = scaler.fit_transform(customer_df[['Total_Bill_Size']])
customer_df.head()

,CustomerID,Total_Bill_Size,First_Purchase,Last_Purchase,Most_Common_Location,Top_Item,Purchase_Interval_Days,Total_Bill_Size_Scaled
0,12346.0,0.00,2011-01-18 10:01:00,2011-01-18 10:17:00,United Kingdom,23166,0,-0.231001
1,12347.0,4310.00,2010-12-07 14:57:00,2011-12-07 15:52:00,Iceland,22375,365,0.293432
2,12348.0,1797.24,2010-12-16 19:09:00,2011-09-25 13:13:00,Finland,POST,282,-0.012316
3,12349.0,1757.55,2011-11-21 09:51:00,2011-11-21 09:51:00,Italy,23112,0,-0.017146
4,12350.0,334.40,2011-02-02 16:01:00,2011-02-02 16:01:00,Norway,21908,0,-0.190312


## Apply K-Means

In [11]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)  # You may want to adjust the number of clusters based on your analysis
customer_df['Cluster'] = kmeans.fit_predict(customer_df[['Total_Bill_Size_Scaled']])
customer_df.head()

,CustomerID,Total_Bill_Size,First_Purchase,Last_Purchase,Most_Common_Location,Top_Item,Purchase_Interval_Days,Total_Bill_Size_Scaled,Cluster
0,12346.0,0.00,2011-01-18 10:01:00,2011-01-18 10:17:00,United Kingdom,23166,0,-0.231001,0
1,12347.0,4310.00,2010-12-07 14:57:00,2011-12-07 15:52:00,Iceland,22375,365,0.293432,0
2,12348.0,1797.24,2010-12-16 19:09:00,2011-09-25 13:13:00,Finland,POST,282,-0.012316,0
3,12349.0,1757.55,2011-11-21 09:51:00,2011-11-21 09:51:00,Italy,23112,0,-0.017146,0
4,12350.0,334.40,2011-02-02 16:01:00,2011-02-02 16:01:00,Norway,21908,0,-0.190312,0


## Cluster Summary

In [12]:
# Function to summarize cluster information with improved readability
def summarize_cluster_info(clustered_df):
    for i in range(kmeans.n_clusters):
        cluster_data = clustered_df[clustered_df['Cluster'] == i]
        print(f"\nCluster {i} Summary:")

        # Customer count in each cluster
        customer_count = len(cluster_data)
        print(f"Number of Customers in the cluster: {customer_count}")

        # Average spend in each cluster
        avg_spend = cluster_data['Total_Bill_Size'].mean()
        print(f"Average Spend: ${avg_spend:.2f}")

        # Top 3 Locations with counts
        top_locations = cluster_data['Most_Common_Location'].value_counts().head(3)
        print("Top 3 Locations:")
        for location, count in top_locations.items():
            print(f"{location}: {count} customers")

        # Top 3 Items with counts
        top_items = cluster_data['Top_Item'].value_counts().head(3)
        print("Top 3 Item Codes:")
        for item, count in top_items.items():
            print(f"{item}: {count} times purchased")

# Call the function to display the summary
summarize_cluster_info(customer_df)


Cluster 0 Summary:
Number of Customers in the cluster: 4346
Average Spend: $1434.20
Top 3 Locations:
United Kingdom: 3929 customers
Germany: 95 customers
France: 87 customers
Top 3 Item Codes:
POST: 127 times purchased
85123A: 92 times purchased
22423: 90 times purchased

Cluster 1 Summary:
Number of Customers in the cluster: 3
Average Spend: $241136.56
Top 3 Locations:
United Kingdom: 2 customers
Netherlands: 1 customers
Top 3 Item Codes:
22629: 1 times purchased
22470: 1 times purchased
22189: 1 times purchased

Cluster 2 Summary:
Number of Customers in the cluster: 23
Average Spend: $58418.75
Top 3 Locations:
United Kingdom: 19 customers
EIRE: 2 customers
Australia: 1 customers
Top 3 Item Codes:
22197: 4 times purchased
C2: 2 times purchased
22630: 1 times purchased
